In [1]:
import numpy as np
import pandas as pd

def join_data(files):
    df = pd.read_csv(files[0])
    df.columns = ['Trip Duration','Start Time','Stop Time','Start Station ID','Start Station Name','Start Station Latitude','Start Station Longitude','End Station ID','End Station Name','End Station Latitude','End Station Longitude','Bike ID','User Type','Birth Year','Gender']
    for i in range(1, len(files)):
        temp = pd.read_csv(files[i])
        temp.columns = ['Trip Duration','Start Time','Stop Time','Start Station ID','Start Station Name','Start Station Latitude','Start Station Longitude','End Station ID','End Station Name','End Station Latitude','End Station Longitude','Bike ID','User Type','Birth Year','Gender']
        df = df.append(temp, ignore_index=True)
    return df

In [2]:
list_of_files = ['201701-citibike-tripdata.csv', '201702-citibike-tripdata.csv', '201703-citibike-tripdata.csv', '201704-citibike-tripdata.csv', '201705-citibike-tripdata.csv', '201706-citibike-tripdata.csv', '201707-citibike-tripdata.csv', '201708-citibike-tripdata.csv', '201709-citibike-tripdata.csv', '201710-citibike-tripdata.csv', '201711-citibike-tripdata.csv', '201712-citibike-tripdata.csv']

data = join_data(list_of_files)

In [3]:
data.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,680,2017-01-01 00:00:21,2017-01-01 00:11:41,3226,W 82 St & Central Park West,40.782750,-73.971370,3165,Central Park West & W 72 St,40.775794,-73.976206,25542,Subscriber,1965.0,2
1,1282,2017-01-01 00:00:45,2017-01-01 00:22:08,3263,Cooper Square & E 7 St,40.729236,-73.990868,498,Broadway & W 32 St,40.748549,-73.988084,21136,Subscriber,1987.0,2
2,648,2017-01-01 00:00:57,2017-01-01 00:11:46,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,18147,Customer,NaN,0
3,631,2017-01-01 00:01:10,2017-01-01 00:11:42,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,21211,Customer,NaN,0
4,621,2017-01-01 00:01:25,2017-01-01 00:11:47,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,26819,Customer,NaN,0


# Create a table of Station ID and its Latitude and Longitude

In [32]:
start_stations = data[['Start Station ID', 'Start Station Latitude', 'Start Station Longitude']].drop_duplicates('Start Station ID')

In [33]:
start_stations.rename(index=str, columns={'Start Station ID': 'Station ID', 'Start Station Latitude': 'Latitude', 'Start Station Longitude': 'Longitude'}, inplace=True)

In [34]:
end_stations = data[['End Station ID', 'End Station Latitude', 'End Station Longitude']].drop_duplicates('End Station ID')

In [35]:
end_stations.rename(index=str, columns={'End Station ID': 'Station ID', 'End Station Latitude': 'Latitude', 'End Station Longitude': 'Longitude'}, inplace=True)

In [38]:
# This returns the table of stations desired
stations = start_stations.append(end_stations, ignore_index=True).drop_duplicates('Station ID')

In [150]:
stations.sort_values(by='Station ID', inplace=True)

In [152]:
# Save the info in a csv file
stations.to_csv('Stations and Locations.csv', index=False)

# Data transformation

In [5]:
data = data[['Start Time', 'Stop Time', 'Start Station ID', 'End Station ID']]

In [6]:
def get_date(df):
    return df['Start Time'].split()[0]

In [7]:
date_col = data.apply(get_date, axis=1)

In [8]:
data['Date'] = date_col

In [9]:
def get_start_hour(df):
    return df['Start Time'].split()[1].split(':')[0]

In [ ]:
start_hour_col = data.apply(get_start_hour, axis=1)

In [61]:
data['Start Hour'] = start_hour_col

/Users/zexi_ye/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
data.drop(columns=['Start Time', 'Stop Time'], inplace=True)

/Users/zexi_ye/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [92]:
grp_start = data.groupby([data['Start Hour'], data['Date'], data['End Station ID']], as_index=False)

In [104]:
start_data = grp_start.agg('count')

In [109]:
start_data.rename(index=str, columns={'Start Station ID': 'Station ID', 'End Station ID': 'Departures'}, inplace=True)

In [111]:
start_data

,Station ID,Date,Start Hour,Departures
0,72,2017-01-01,02,1
1,72,2017-01-01,03,1
2,72,2017-01-01,05,2
3,72,2017-01-01,08,2
4,72,2017-01-01,09,1
5,72,2017-01-01,10,2
6,72,2017-01-01,12,5
7,72,2017-01-01,13,6
8,72,2017-01-01,15,7
9,72,2017-01-01,16,6


In [112]:
grp_end = data.groupby([data['Start Hour'], data['Date'], data['End Station ID']], as_index=False)

In [113]:
end_data = grp_end.agg('count')

In [115]:
end_data.rename(index=str, columns={'End Station ID': 'Station ID', 'Start Station ID': 'Arrivals'}, inplace=True)

In [132]:
combined_data = start_data.merge(end_data, how='outer', on=['Station ID', 'Date', 'Start Hour'])

In [137]:
combined_data.sort_values(by=['Station ID', 'Date', 'Start Hour'], inplace=True)

In [142]:
combined_data.fillna(0, inplace=True)

In [146]:
combined_data.Departures = combined_data.Departures.astype(int)
combined_data.Arrivals = combined_data.Arrivals.astype(int)

In [148]:
combined_data.to_csv('Citibike Trip Count Data 2017.csv', index=False)